In [1]:
import pandas as pd
import numpy as np
import os
import tqdm
import shutil

In [2]:
dataset = 'emma' # onion or emma
input_data_dir = f'../data/'

features = {
    'emotion_embeddings': ['id_emma.tsv', 'id_gems.tsv'],
    'audio_embeddings': ['id_maest.tsv', 'id_jukebox.tsv', 'id_musicnn.tsv'],
    'textual_embeddings': ['id_bert.tsv', 'id_lyrics_word2vec.tsv'],
    'visual_embeddings': ['id_resnet.tsv', 'id_vgg19.tsv', 'id_incp.tsv'],
}

output_dir = f'../dataset/{dataset}/'

os.makedirs(output_dir, exist_ok=True)

In [3]:
df_interactions = pd.read_csv(f'../data/userid_trackid_timestamp_{dataset}.tsv', sep='\t')
    
df_interactions['rating'] = 5
sample_items = df_interactions['track_id'].unique()

# map item and users to numbers
df_interactions['user_id_int'] = df_interactions['user_id'].astype('category').cat.codes
df_interactions['track_id_int'] = df_interactions['track_id'].astype('category').cat.codes


print(df_interactions.shape)
df_interactions.head()

(52946, 8)


,user_id,track_id,timestamp,year,month,rating,user_id_int,track_id_int
0,2,9CHzAeSHv9tWaIXW,2019-12-22 06:25:40,2019,2019-12,5,0,33
1,6,RbM2Pe6nEHAk99L6,2019-01-21 20:48:25,2019,2019-01,5,1,116
2,6,sEy3cQGc1GEYMfo3,2019-01-21 21:38:25,2019,2019-01,5,1,225
3,6,yfm7RER1PWTqgjIp,2019-02-01 17:51:25,2019,2019-02,5,1,261
4,6,FpSHrgYPnwJvVuZE,2019-04-02 01:45:32,2019,2019-04,5,1,64


In [4]:
# track ids from feature file
df = pd.read_csv(input_data_dir + features['emotion_embeddings'][1], sep='\t')
feat_track_ids = df['id'].values
feat_track_ids

array(['04q3VppIQEET5rzy', '07xF9Q0K1t3ist7K', '0JbYcELIqpMPvAHk', ...,
       'zzyyPUs7hC9Nz2e1', 'zzz0n04uuTUA7fNh', 'zzzj3LYaZtYtbzSr'],
      dtype=object)

In [5]:
# remove interactions with items not in the feature files
df_interactions = df_interactions[df_interactions['track_id'].isin(feat_track_ids)]
df_interactions.shape

(52946, 8)

In [6]:
#df_interactions[['user_id', 'track_id', 'rating', 'timestamp']].to_csv(os.path.join(output_dir, 'interactions.tsv'), index=False, sep='\t', header=False)
df_interactions[['user_id_int', 'track_id_int', 'rating', 'timestamp']].to_csv(os.path.join(output_dir, 'interactions.tsv'), index=False, sep='\t', header=False)

In [7]:
item_id_map = df_interactions[['track_id', 'track_id_int']].drop_duplicates()
item_id_map.set_index('track_id', inplace=True)
item_id_map.head()

,track_id_int
track_id,
9CHzAeSHv9tWaIXW,33
RbM2Pe6nEHAk99L6,116
sEy3cQGc1GEYMfo3,225
yfm7RER1PWTqgjIp,261
FpSHrgYPnwJvVuZE,64


In [8]:
del df_interactions

In [9]:
for feature_type in features:

    for file in features[feature_type]:
        if dataset == 'onion' and 'emma' in file:
            continue

        print('Processing', file)
        df = pd.read_csv(os.path.join(input_data_dir, file), index_col=0, sep='\t')
        df = df.merge(item_id_map, left_index=True, right_index=True)
        feature_name = file.split('.')[0].split('_')[1]
            
        folder_path = os.path.join(output_dir, feature_type, feature_name)

        if os.path.exists(folder_path):
            shutil.rmtree(folder_path)
        os.makedirs(folder_path, exist_ok=True)

        for index, row in tqdm.tqdm(df.iterrows()):
            np.save(os.path.join(folder_path, str(int(row['track_id_int'])) + '.npy'), row.values[:-1])

Processing id_emma.tsv


267it [00:01, 146.31it/s]


Processing id_gems.tsv


267it [00:01, 144.92it/s]


Processing id_maest.tsv


...

267it [00:02, 97.87it/s] 


Processing id_vgg19.tsv


267it [00:04, 66.00it/s]


Processing id_incp.tsv


267it [00:02, 97.74it/s]
